# Part IV — Professional Django  
## 19. Security (OWASP‑Aligned) — Harden Django the Way Professionals Do

Security in Django is not “add one setting and you’re done.” It’s a system:

- secure defaults (Django already helps a lot)
- correct configuration per environment
- correct use of framework features (CSRF, escaping, ORM)
- safe patterns for auth, uploads, and permissions
- operational discipline (patching, secrets, logging, monitoring)

This chapter is written so that, after finishing it, you can do a real “security
review pass” on any Django project and know what to fix.

---

## 19.0 Learning Outcomes

By the end of this chapter you should be able to:

1. Explain Django’s main threat areas using OWASP categories (CSRF, XSS, auth,
   injection, insecure config, sensitive data exposure, broken access control).
2. Configure **production-safe settings**:
   - `DEBUG=False`, correct `ALLOWED_HOSTS`
   - secure cookies and session settings
   - HTTPS/HSTS settings (with proxy considerations)
   - CSRF trusted origins for modern deployments
3. Implement safe patterns for:
   - CSRF protection in forms and AJAX
   - XSS prevention and safe HTML rendering
   - SQL injection prevention (ORM + safe raw SQL)
4. Secure file uploads (size/type/path, private media concept).
5. Add security headers (CSP, X-Frame-Options, HSTS, Referrer-Policy, etc.).
6. Add rate limiting/throttling conceptually (and know where Django ends and extra
   tooling begins).
7. Write tests that validate critical security behaviors:
   - CSRF rejects missing token
   - security headers exist (when configured)
   - login-required redirects/403 for unauthorized actions
8. Produce a practical security checklist you can reuse.

---

## 19.1 The Security Mindset (What “Secure” Means in a Web App)

### 19.1.1 Security goals
- **Confidentiality**: only authorized people see data.
- **Integrity**: only authorized changes happen.
- **Availability**: the system stays usable under expected load and common abuse.

### 19.1.2 Threat model (simplified but practical)
Assume attackers can:
- send any HTTP request they want
- manipulate query params, headers, cookies, forms
- automate requests at high volume
- try to guess IDs (enumeration)
- upload malicious files
- attempt credential stuffing (reusing leaked passwords)
- attempt XSS/CSRF/injection payloads

Your job is to:
- ensure the server enforces rules (not the UI)
- keep secrets secret
- validate and constrain inputs
- minimize “blast radius” of mistakes (defense in depth)

---

## 19.2 Security Starts with Configuration (The “Insecure Defaults” Problem)

OWASP has a category “Security Misconfiguration.” In Django, a huge portion of real
incidents are misconfig, not code.

### 19.2.1 The non-negotiables
In production you must have:
- `DEBUG = False`
- a secret `SECRET_KEY` (not committed)
- correct `ALLOWED_HOSTS`
- HTTPS + secure cookies
- proper CSRF origin configuration (especially with modern domain setups)
- correct error handling and logging (no sensitive info leakage)

---

## 19.3 Secrets Management: `SECRET_KEY` and Environment Variables

### 19.3.1 What `SECRET_KEY` does
Django uses `SECRET_KEY` for cryptographic signing (examples):
- session cookies (when using signed cookies)
- password reset tokens
- CSRF token generation/signing aspects
- signing utilities (`django.core.signing`)

If an attacker gets your `SECRET_KEY`, they can often:
- forge signed values
- compromise reset tokens
- undermine trust boundaries

### 19.3.2 Industry standard: do not store secrets in code
Instead, use environment variables.

In `config/settings.py` (or a dedicated settings module later):

```python
import os

SECRET_KEY = os.environ["DJANGO_SECRET_KEY"]
```

This will crash if missing (good). You want a loud failure, not a silent insecure
default.

### 19.3.3 Local development convenience without committing secrets
Common patterns:
- `.env` file (not committed) + a loader package (optional)
- export env vars in your shell
- use container secrets in Docker/Kubernetes

If you use `.env`, ensure `.env` is in `.gitignore`.

---

## 19.4 Debug Mode: Why `DEBUG=True` Is Dangerous

### 19.4.1 What `DEBUG=True` leaks
In production, `DEBUG=True` can expose:
- detailed stack traces
- environment variables
- settings values
- installed apps
- SQL queries in some cases
- code paths and internal structure

This information helps attackers.

### 19.4.2 Correct pattern
```python
DEBUG = os.environ.get("DJANGO_DEBUG", "false").lower() == "true"
```

Then in production: `DJANGO_DEBUG=false`.

---

## 19.5 `ALLOWED_HOSTS` and Host Header Attacks (Often Missed)

### 19.5.1 What is the Host header?
The client sends:

```text
Host: example.com
```

Django uses it for:
- building absolute URLs in some contexts
- security decisions (when configured)
- determining which site is being accessed

### 19.5.2 What’s the attack?
If `ALLOWED_HOSTS` is misconfigured (too permissive), an attacker can send:

```text
Host: evil.com
```

This can lead to:
- cache poisoning
- password reset links generated with attacker-controlled host (in some setups)
- incorrect redirects and security boundary confusion

### 19.5.3 Correct configuration
In production:

```python
ALLOWED_HOSTS = ["example.com", "www.example.com"]
```

If you have subdomains:

```python
ALLOWED_HOSTS = [".example.com"]
```

Do not use `["*"]` in production.

---

## 19.6 HTTPS, Proxies, and “Why It Works Locally But Breaks in Production”

Most production Django apps run behind a reverse proxy/load balancer that terminates
TLS. Django receives plain HTTP from the proxy even though the user used HTTPS.

### 19.6.1 Enforcing HTTPS
Typical settings:

```python
SECURE_SSL_REDIRECT = True
```

Meaning:
- if request is not secure, redirect to HTTPS

### 19.6.2 Proxy header configuration
If TLS is terminated upstream, Django needs to know the original scheme. Many setups
use:

```python
SECURE_PROXY_SSL_HEADER = ("HTTP_X_FORWARDED_PROTO", "https")
```

This tells Django:
- treat request as secure when the proxy set `X-Forwarded-Proto: https`

**Important:** Only set this if your proxy reliably sets that header and you trust
it. Otherwise attackers could spoof it.

### 19.6.3 HSTS (Strict Transport Security)
HSTS tells browsers “always use HTTPS for this domain.”

```python
SECURE_HSTS_SECONDS = 31536000  # 1 year
SECURE_HSTS_INCLUDE_SUBDOMAINS = True
SECURE_HSTS_PRELOAD = True
```

**Industry caution:** start with a smaller value (e.g., 1 day) during rollout, then
increase. HSTS can lock you into HTTPS—good, but you must be ready.

---

## 19.7 Cookies and Session Security (High Impact, Low Effort)

Your app uses sessions for authentication. Sessions ride on cookies.

### 19.7.1 Key session/csrf cookie settings (production)
Typical baseline:

```python
SESSION_COOKIE_HTTPONLY = True
SESSION_COOKIE_SECURE = True
SESSION_COOKIE_SAMESITE = "Lax"

CSRF_COOKIE_SECURE = True
CSRF_COOKIE_SAMESITE = "Lax"
```

Explain each:

- `HttpOnly` prevents JavaScript from reading the session cookie (reduces impact of XSS).
- `Secure` ensures cookies are sent only over HTTPS (prevents leakage over HTTP).
- `SameSite=Lax` reduces CSRF risk for many common flows while keeping normal
  navigation functional.

### 19.7.2 Session lifetime
Default session age might be long. You can tune:

```python
SESSION_COOKIE_AGE = 60 * 60 * 24 * 7  # 7 days
SESSION_EXPIRE_AT_BROWSER_CLOSE = False
```

For highly sensitive apps, reduce age and consider idle timeouts (more advanced).

### 19.7.3 CSRF cookie HttpOnly?
By default, CSRF cookie is often readable by JS because some architectures read it
to set AJAX headers. If you don’t need JS to read it, you can tighten it, but only
after understanding your frontend needs.

---

## 19.8 CSRF (Cross‑Site Request Forgery) — Deep, Practical, Django‑Specific

### 19.8.1 The real risk in Django apps
If you rely on cookie-based auth (session), your browser automatically sends those
cookies with requests. That is what enables CSRF.

### 19.8.2 Django’s protection model
Django’s CSRF middleware:
- sets a CSRF cookie (token)
- expects the token to be present in unsafe requests (usually in form body)
- validates token matches expected value
- blocks if missing/invalid (403)

### 19.8.3 Correct usage in HTML forms
Every POST form must include:

```django
<form method="post">
  {% csrf_token %}
  ...
</form>
```

### 19.8.4 Correct usage for AJAX / fetch (common pattern)
If you make POST requests using JavaScript, include `X-CSRFToken` header.

A typical approach:
1. Read CSRF token from cookie (or from DOM).
2. Send it in header.

JavaScript example (classic approach):

```javascript
function getCookie(name) {
  const parts = document.cookie.split(";").map((c) => c.trim());
  for (const part of parts) {
    if (part.startsWith(name + "=")) {
      return decodeURIComponent(part.slice(name.length + 1));
    }
  }
  return null;
}

async function postJson(url, data) {
  const csrftoken = getCookie("csrftoken");
  const res = await fetch(url, {
    method: "POST",
    headers: {
      "Content-Type": "application/json",
      "X-CSRFToken": csrftoken,
    },
    body: JSON.stringify(data),
    credentials: "same-origin",
  });
  return res.json();
}
```

Important notes:
- `credentials: "same-origin"` ensures cookies are sent (needed for session auth).
- CSRF token header must be present.

### 19.8.5 CSRF + modern deployments: `CSRF_TRUSTED_ORIGINS`
If your site is served from multiple origins (e.g., behind a CDN, or using
`https://app.example.com` + `https://api.example.com`), you often must configure:

```python
CSRF_TRUSTED_ORIGINS = [
    "https://example.com",
    "https://www.example.com",
]
```

This is commonly required when:
- your reverse proxy changes host/origin behavior
- you use HTTPS and strict origin checks

### 19.8.6 Testing CSRF correctly (so you know you didn’t disable it)
Django’s test client disables CSRF checks by default. To test CSRF, you must enable
checks.

Create `tests/test_csrf.py`:

```python
from django.test import Client, TestCase
from django.urls import reverse


class CsrfTests(TestCase):
    def test_post_without_csrf_is_rejected(self):
        client = Client(enforce_csrf_checks=True)

        url = reverse("pages:set_theme")  # POST endpoint you created earlier
        response = client.post(url, {"theme": "dark"})

        self.assertEqual(response.status_code, 403)
```

Now you know CSRF middleware is actually protecting you.

---

## 19.9 XSS (Cross‑Site Scripting) — How It Happens in Django and How to Avoid It

### 19.9.1 Django templates escape by default (huge protection)
If you render:

```django
<p>{{ user_input }}</p>
```

and `user_input` contains `<script>...</script>`, Django escapes it and it becomes
text, not executable code.

### 19.9.2 How developers accidentally reintroduce XSS
#### Mistake A: Using `|safe` on untrusted content
Example:

```django
<div>{{ comment.body|safe }}</div>
```

If comment body comes from a user, you have an XSS vulnerability.

#### Mistake B: Using `mark_safe` in Python
Example (dangerous if `html` contains untrusted input):

```python
from django.utils.safestring import mark_safe

return mark_safe(html)
```

#### Mistake C: Injecting untrusted values into JavaScript contexts
Example:

```django
<script>
  const name = "{{ user_input }}";
</script>
```

If `user_input` contains quotes and JS code, you can break out and execute code.

### 19.9.3 Safe HTML construction: `format_html`
If you must construct HTML in Python (e.g., admin display), use `format_html`:

```python
from django.utils.html import format_html

def link_to_article(article):
    return format_html(
        '<a href="{}">{}</a>',
        article.get_absolute_url(),
        article.title,
    )
```

`format_html` escapes arguments but treats the template string as safe.

### 19.9.4 Handling rich text safely (real-world note)
If you allow users to write rich HTML (bold, links), you need sanitization:
- remove scripts
- restrict tags/attributes
- handle URLs safely

This typically requires a dedicated HTML sanitizer library and careful policy.
Do not “just use safe.”

---

## 19.10 SQL Injection and ORM Safety (Django Helps, But You Can Still Break It)

### 19.10.1 Why ORM is safe by default
When you do:

```python
Article.objects.filter(slug=user_input)
```

Django uses parameterized queries. The SQL structure is separate from the values,
so user input cannot “rewrite” the query.

### 19.10.2 How you can still introduce SQL injection
#### Dangerous pattern: string formatting raw SQL
Bad:

```python
from django.db import connection

def unsafe_search(q):
    with connection.cursor() as cursor:
        cursor.execute(f"SELECT * FROM articles_article WHERE title LIKE '%{q}%'")
        return cursor.fetchall()
```

If `q` contains SQL syntax, you’re compromised.

### 19.10.3 Safe raw SQL (parameterized)
Good:

```python
from django.db import connection

def safe_search(q):
    with connection.cursor() as cursor:
        cursor.execute(
            "SELECT id, title FROM articles_article WHERE title LIKE %s",
            [f"%{q}%"],
        )
        return cursor.fetchall()
```

Key idea:
- SQL string stays constant
- values passed separately

### 19.10.4 Prefer ORM unless you truly need raw SQL
If you can do it in ORM, do it:

```python
Article.objects.filter(title__icontains=q)
```

Raw SQL is for:
- complex performance cases
- database-specific features not exposed by ORM
- reporting queries that are hard to express otherwise

If you use raw SQL, keep it:
- parameterized
- tested
- reviewed carefully

---

## 19.11 Broken Access Control (The #1 Real Web App Vulnerability)

Most serious business-impact bugs in Django apps are:
- missing authorization checks
- incorrect scoping
- relying on UI-only restrictions

### 19.11.1 Correct pattern: scope in the query
From Project 2 tasks:

```python
task = get_object_or_404(Task, id=task_id, organization=org)
```

This prevents ID guessing from leaking cross-org data.

### 19.11.2 Correct pattern: enforce object-level authorization
From your articles:

- owner or staff can edit
- others get 403/404

Never rely on hiding the edit button in template; enforce in view.

### 19.11.3 Test access control explicitly
Security tests are just normal tests that assert correct status codes.

Examples:
- outsider gets 404 for org pages
- non-owner gets 403 for edit
- anonymous gets redirected to login

These tests are extremely high value.

---

## 19.12 File Upload Security (Commonly Exploited if Done Wrong)

Uploads are dangerous because they:
- accept untrusted bytes
- can cause storage issues (large files)
- can be used to host malicious content if served publicly
- can trigger parser vulnerabilities (images/PDFs)

### 19.12.1 Mandatory controls for uploads
1. **Size limits**
2. **Type validation**
3. **Storage location control**
4. **Public vs private decision**
5. **Anti-path traversal** (don’t trust filenames; use Django storage)

You already added basic size/type checks in `clean_cover_image`. Good.

### 19.12.2 Global upload limits (Django settings)
These prevent huge memory usage in request parsing:

```python
DATA_UPLOAD_MAX_MEMORY_SIZE = 10 * 1024 * 1024  # 10MB
FILE_UPLOAD_MAX_MEMORY_SIZE = 5 * 1024 * 1024   # 5MB
```

Meaning:
- limit how much request data Django will hold in memory
- larger files go to temp files on disk (depending on handlers)

### 19.12.3 Serving uploads: public vs private
- Public media: images on a blog post cover
- Private media: invoices, ID documents

If media is private, you should **not** serve it directly from `/media/` publicly.
You need:
- authenticated view that checks permissions, then streams file
- or signed URLs from storage (S3) for time-limited access

This is a major architectural decision.

---

## 19.13 Security Headers (Defense‑in‑Depth for Browsers)

Headers don’t replace secure code, but they reduce impact of bugs.

### 19.13.1 Common headers and what they do

- `X-Frame-Options: DENY` or `SAMEORIGIN`
  - prevents clickjacking (embedding your site in an iframe)
- `Content-Security-Policy (CSP)`
  - reduces XSS impact by restricting script sources
- `Referrer-Policy`
  - controls what referrer info is sent to other sites
- `Permissions-Policy`
  - restricts browser APIs (camera, mic, geolocation)
- `Strict-Transport-Security` (HSTS)
  - forces HTTPS

Django already provides `XFrameOptionsMiddleware` and `SecurityMiddleware`. CSP often
requires custom middleware or a package.

### 19.13.2 Add a simple security headers middleware (workbook implementation)
Create `config/security_headers.py`:

```python
from django.http import HttpRequest, HttpResponse


class SecurityHeadersMiddleware:
    def __init__(self, get_response):
        self.get_response = get_response

    def __call__(self, request: HttpRequest) -> HttpResponse:
        response = self.get_response(request)

        # Clickjacking protection (Django also has XFrameOptionsMiddleware)
        response.setdefault("X-Frame-Options", "DENY")

        # MIME sniffing protection
        response.setdefault("X-Content-Type-Options", "nosniff")

        # Reduce referrer leakage
        response.setdefault("Referrer-Policy", "strict-origin-when-cross-origin")

        # Basic permissions policy (tighten as needed)
        response.setdefault(
            "Permissions-Policy",
            "geolocation=(), microphone=(), camera=()",
        )

        return response
```

Register it in `MIDDLEWARE` (ideally near the top or near security middleware; be
consistent). Example:

```python
MIDDLEWARE = [
    "django.middleware.security.SecurityMiddleware",
    "config.security_headers.SecurityHeadersMiddleware",
    # ...
]
```

### 19.13.3 CSP (Content Security Policy) — do it intentionally
CSP is powerful but can break scripts/styles if you use inline JS/CSS.

A strict CSP might look like:

```text
Content-Security-Policy: default-src 'self'; img-src 'self' data:; style-src 'self'; script-src 'self'
```

However:
- if you use external CDNs, you must whitelist them
- if you use inline scripts, you must use nonces/hashes (advanced)

For many Django sites, adding CSP is a later hardening step. The key is:
- understand it
- apply it gradually
- test thoroughly

---

## 19.14 Authentication Hardening (Passwords, Brute Force, 2FA Concepts)

### 19.14.1 Password validation (Django supports this)
In `config/settings.py`, enable and tune validators:

```python
AUTH_PASSWORD_VALIDATORS = [
    {"NAME": "django.contrib.auth.password_validation.UserAttributeSimilarityValidator"},
    {"NAME": "django.contrib.auth.password_validation.MinimumLengthValidator",
     "OPTIONS": {"min_length": 10}},
    {"NAME": "django.contrib.auth.password_validation.CommonPasswordValidator"},
    {"NAME": "django.contrib.auth.password_validation.NumericPasswordValidator"},
]
```

This reduces weak passwords.

### 19.14.2 Brute force / credential stuffing protection
Django does not include full login rate limiting out of the box.

Industry approach:
- rate limit at reverse proxy (Nginx, CDN, WAF)
- and/or use a Django package for login attempt tracking/lockout
- and monitor logs

Even without packages, you should:
- keep login endpoints protected by strong passwords
- consider MFA/SSO for admin/staff
- add monitoring for repeated failures

### 19.14.3 Admin hardening
- restrict admin access (VPN/IP allowlist if possible)
- enforce MFA for staff accounts (often via external SSO/MFA solution)
- keep superusers minimal

---

## 19.15 CORS vs CSRF (Common Confusion)

### CORS
- browser rule about which origins can read responses via JS
- configured via response headers (`Access-Control-Allow-Origin`, etc.)
- relevant for APIs consumed by frontend apps on different domains

### CSRF
- protects against cross-site *state-changing requests* that leverage cookies

Important:
- CORS does not automatically protect you from CSRF
- if you use cookies for auth across origins, you need a careful CSRF strategy

If you build DRF APIs for SPAs, you will revisit this in the API security chapters.

---

## 19.16 Dependency and Patch Security (Often Overlooked)

### 19.16.1 Why this matters
Most real-world compromises involve known vulnerabilities in dependencies.

Professional practice:
- pin versions
- update regularly
- use a vulnerability scanner in CI (tooling varies)
- subscribe to security advisories for Django and key libs

### 19.16.2 Minimum habit
- keep Django up to date (especially security releases)
- avoid abandoned packages
- audit dependencies periodically

---

## 19.17 LAB: Build a “Production Settings” Layer (Safe Defaults)

You can keep a single settings file with env switches, or split settings modules.
A common industry pattern is split settings.

### 19.17.1 Create `config/settings/` package
Restructure:

```text
config/settings/
  __init__.py
  base.py
  dev.py
  prod.py
```

#### `config/settings/base.py` (shared settings)
Move most current settings here.

Add secure defaults that are environment-agnostic.

#### `config/settings/dev.py`
```python
from .base import *

DEBUG = True
ALLOWED_HOSTS = ["127.0.0.1", "localhost"]

EMAIL_BACKEND = "django.core.mail.backends.console.EmailBackend"
```

#### `config/settings/prod.py`
```python
import os
from .base import *

DEBUG = False

SECRET_KEY = os.environ["DJANGO_SECRET_KEY"]

ALLOWED_HOSTS = os.environ["DJANGO_ALLOWED_HOSTS"].split(",")

CSRF_TRUSTED_ORIGINS = os.environ.get("DJANGO_CSRF_TRUSTED_ORIGINS", "").split(",")

SESSION_COOKIE_SECURE = True
CSRF_COOKIE_SECURE = True
SESSION_COOKIE_HTTPONLY = True
SESSION_COOKIE_SAMESITE = "Lax"
CSRF_COOKIE_SAMESITE = "Lax"

SECURE_SSL_REDIRECT = True
SECURE_HSTS_SECONDS = 31536000
SECURE_HSTS_INCLUDE_SUBDOMAINS = True
SECURE_HSTS_PRELOAD = True

# Uncomment only if behind a proxy that sets X-Forwarded-Proto correctly
# SECURE_PROXY_SSL_HEADER = ("HTTP_X_FORWARDED_PROTO", "https")
```

Then update `manage.py`, `wsgi.py`, `asgi.py` to point to dev by default or use env
var:

- Set `DJANGO_SETTINGS_MODULE=config.settings.dev` in dev
- In production environment, set `DJANGO_SETTINGS_MODULE=config.settings.prod`

This is a standard pattern; it makes “dev vs prod” differences explicit and reviewable.

---

## 19.18 Tests for Security Headers (So You Know They Exist)

Create `tests/test_security_headers.py` (pytest) or Django TestCase.

Django TestCase example:

```python
from django.test import TestCase, override_settings


class SecurityHeadersTests(TestCase):
    @override_settings(DEBUG=False)
    def test_security_headers_present(self):
        response = self.client.get("/healthz/")
        self.assertEqual(response.status_code, 200)

        self.assertEqual(response.headers.get("X-Content-Type-Options"), "nosniff")
        self.assertIn(
            response.headers.get("Referrer-Policy", ""),
            {"strict-origin-when-cross-origin", "no-referrer"},
        )
        self.assertTrue("Permissions-Policy" in response.headers)
```

Notes:
- You may decide to always set these headers even in debug. That’s fine.
- For HTTPS-only headers (HSTS), you’ll see them only when SecurityMiddleware is configured and request is secure in some environments.

---

## 19.19 A Practical Security Checklist (Use This Every Time)

### Configuration
- [ ] `DEBUG=False` in production
- [ ] `SECRET_KEY` from environment; not committed
- [ ] `ALLOWED_HOSTS` locked to real domains
- [ ] `CSRF_TRUSTED_ORIGINS` set if needed for your deployment topology
- [ ] HTTPS enforced (`SECURE_SSL_REDIRECT`, proxy settings correct)
- [ ] HSTS enabled after validation
- [ ] secure cookies (`SESSION_COOKIE_SECURE`, `CSRF_COOKIE_SECURE`, SameSite)
- [ ] strong password validators enabled

### App-level protections
- [ ] CSRF tokens present in all POST forms
- [ ] templates do not use `safe` on untrusted content
- [ ] raw SQL (if any) is parameterized
- [ ] all sensitive views enforce authorization server-side
- [ ] object scoping enforced in DB queries (org/user boundaries)
- [ ] uploads have size/type limits and correct storage handling

### Headers
- [ ] clickjacking protection (X-Frame-Options)
- [ ] nosniff
- [ ] Referrer-Policy
- [ ] Permissions-Policy
- [ ] CSP planned/implemented (optional but recommended as maturity grows)

### Operations
- [ ] dependencies pinned and updated regularly
- [ ] vulnerability scanning in CI (or scheduled audits)
- [ ] admin access restricted; MFA/SSO for staff if possible
- [ ] logs don’t leak secrets; error pages don’t leak internals

---

## 19.20 Exercises (Do These Before Proceeding)

1. Add CSRF tests for at least two POST endpoints using `enforce_csrf_checks=True`.
2. Search your templates for `|safe` usage. For each occurrence:
   - explain why it’s safe, or
   - remove it and replace with safe rendering.
3. Add a “security headers middleware” and write tests for header presence.
4. Add global upload size limits in settings and document them in README.
5. Review your Project 2 task scoping:
   - verify every task query includes `organization=org`
   - add a regression test ensuring outsider cannot access tasks by ID.

---

## 19.21 Chapter Summary (What to Retain)

- Django provides strong security primitives, but you must configure and use them correctly.
- Security misconfiguration (DEBUG, hosts, cookies, HTTPS) is a major risk category.
- CSRF is about cookie-auth state changes; always include CSRF tokens for POST.
- XSS is prevented by escaping—don’t disable it with `safe` unless you have sanitized content.
- ORM prevents SQL injection by default; raw SQL must be parameterized.
- Broken access control is the biggest practical risk: scope queries + enforce permissions + test them.
- Uploads and headers are essential defense-in-depth.

---

Next chapter: **Part IV — 20. Performance and Scaling Fundamentals**  
We’ll measure performance properly, eliminate N+1 problems systematically, add
caching correctly, understand indexes in practice, and build a performance checklist
suitable for production reviews.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='18 testing.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='20. performance_and_scaling_fundamentals.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
